# Save GGUF Model from LoRA Checkpoint (Colab)
このノートブックは Google Colab 上で実行し、
Windows などで学習済みの LoRA チェックポイント (例: `checkpoint-886.zip`) を
ベースモデル `unsloth/gpt-oss-20b` に適用して GGUF 形式に変換する手順をまとめています。

## 1. ランタイムの準備- Colab で **GPU** ランタイムに切り替えてから実行してください (T4/A100 等)。- 右上の `接続` → `ランタイム` → `ランタイムのタイプを変更` から GPU を選択します。

In [ ]:
#@title Install dependencies
!pip install --upgrade --quiet unsloth==2025.8.10 peft==0.12.0 accelerate==0.34.2 transformers==4.45.1 safetensors huggingface_hub
!pip install --quiet sentencepiece bitsandbytes

## 2. Google Drive をマウント (任意)チェックポイント ZIP を Google Drive に置いている場合は以下を実行してマウントします。

In [ ]:
#@title Mount Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

## 3. チェックポイントを配置- 以下では `/content/checkpoint-886.zip` に置かれている前提です。- Colab のファイルアップローダや Google Drive からのコピーで同じパスに配置してください。- 複数のチェックポイントがある場合は適宜パスを変更してください。

In [ ]:
#@title Set paths
from pathlib import Path
CHECKPOINT_ZIP = Path('/content/checkpoint-886.zip')  # @param {type:"string"}
WORK_DIR = Path('/content/runs/infer')
OUTPUT_DIR = Path('/content/outputs')
WORK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
#@title Unzip LoRA checkpoint
import zipfile
if not CHECKPOINT_ZIP.exists():
    raise FileNotFoundError(f'Checkpoint zip not found: {CHECKPOINT_ZIP}')
with zipfile.ZipFile(CHECKPOINT_ZIP, 'r') as zf:
    zf.extractall(WORK_DIR)
print('Extracted to:', WORK_DIR)

## 4. UnsLoTH でベースモデル + LoRA をロード- `unsloth/gpt-oss-20b` を 4bit 量子化でロードし、LoRA アダプタを適用します。- `checkpoint_dir` は zip 展開後のフォルダ (例: `runs/infer/checkpoint-886`) を指すようにしてください。

In [ ]:
#@title Load base model and apply LoRA
import torch
from unsloth import FastLanguageModel
from peft import PeftModel

base_model_name = 'unsloth/gpt-oss-20b'
checkpoint_dir = next(WORK_DIR.glob('checkpoint-*'))
print('Using checkpoint dir:', checkpoint_dir)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,
    max_seq_length = 4096,
    load_in_4bit = True,
    dtype = None,
)

model = PeftModel.from_pretrained(model, checkpoint_dir)
model = FastLanguageModel.for_inference(model, dtype=None)
print('Model + LoRA loaded!')

## 5. LoRA をマージして GGUF を出力- 保存先は `OUTPUT_DIR` (`/content/outputs`) 以下に作成されます。- GGUF 形式では `f16` などの量子化形式を選択可能です (VRAM/メモリに合わせて変更)。

In [ ]:
#@title Export merged model (HF + GGUF)merged_dir = OUTPUT_DIR / 'merged_hf'
gguf_dir = OUTPUT_DIR / 'gguf'
merged_dir.mkdir(parents=True, exist_ok=True)
gguf_dir.mkdir(parents=True, exist_ok=True)

print('Saving merged HF model...')
model.save_pretrained_merged(
    merged_dir,
    tokenizer = tokenizer,
    save_method = 'merged_16bit',
)
tokenizer.save_pretrained(merged_dir)

print('Converting to GGUF (f16)...')
model.save_pretrained_gguf(
    gguf_dir,
    tokenizer,
    quantization_method = 'f16',  # 例: 'q8_0', 'q4_k_m' などに変更可能
)
print('Done! Files written to:', gguf_dir)

## 6. 生成テスト (任意)GGUF 出力前にマージ済みモデルで動作確認を行いたい場合は以下を実行してください。

In [ ]:
#@title Quick generation test (optional)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！ 自己紹介してください。"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
).to(model.device)
from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
_ = model.generate(**inputs, max_new_tokens=256, streamer=streamer)